In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [4]:
df17 = df17.apply(pd.to_numeric, errors = 'coerce')
df17.describe().iloc[:, :15]
# 없는 Column = date, codesum, depart, sunrise, sunset, station_nbr

,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure
count,6.307020e+05,630036.000000,630036.000000,628038.00000,0.0,630036.000000,625374.000000,628038.000000,628038.000000,0.0,0.0,0.0,594738.000000,630036.000000,629370.000000
mean,1.367579e+18,73.488372,48.906977,61.45281,NaN,46.250529,53.312034,9.607635,6.060445,NaN,NaN,NaN,0.000056,0.068039,29.348487
std,2.515189e+16,18.809463,18.516844,18.13576,NaN,18.163112,16.052230,12.144894,8.059406,NaN,NaN,NaN,0.001672,0.215609,0.187040
min,1.325376e+18,19.000000,2.000000,12.00000,NaN,-3.000000,9.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,28.740000
25%,1.345766e+18,61.000000,33.000000,47.00000,NaN,31.000000,39.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,29.220000
50%,1.366330e+18,76.000000,50.000000,64.00000,NaN,49.000000,55.000000,1.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,29.330000
75%,1.389226e+18,89.000000,66.000000,77.00000,NaN,63.000000,68.000000,18.000000,12.000000,NaN,NaN,NaN,0.000000,0.005000,29.460000
max,1.414714e+18,114.000000,82.000000,97.00000,NaN,75.000000,78.000000,53.000000,32.000000,NaN,NaN,NaN,0.050000,3.050000,30.060000


In [3]:
df17['store_nbr'].unique()

array([ 9, 18, 23, 26, 31, 34], dtype=int64)

In [5]:
df17_drop_columns = ['date', 'codesum', 'depart', 'sunrise', 'sunset', 'station_nbr']
df17 = df17.drop(columns = df17_drop_columns)

In [6]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df17_columns = df17.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df17_columns:
    if (i == 'resultdir'):
        df17[i].fillna(df17[i].mode()[0], inplace=True)
        print(df17[i].mode()[0])
    else:
        df17[i].fillna(df17[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df17['relative_humility'] = 100*(np.exp((17.625*((df17['dewpoint']-32)/1.8))/(243.04+((df17['dewpoint']-32)/1.8)))/np.exp((17.625*((df17['tavg']-32)/1.8))/(243.04+((df17['tavg']-32)/1.8))))

# 체감온도 계산
df17["windchill"] = 35.74 + 0.6217*df17["tavg"] - 35.75*(df17["avgspeed"]**0.17) + 0.4275*df17["tavg"]*(df17["avgspeed"]**0.17)

df17 = df17[df17['units'] != 0]

18.0


In [7]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df17 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + snowfall + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df17)
result_df17 = model_df17.fit()

print(result_df17.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     2219.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:55:50   Log-Likelihood:                -12164.
No. Observations:               16381   AIC:                         2.441e+04
Df Residuals:                   16340   BIC:                         2.473e+04
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]          0.0093      0

In [8]:
model_df17 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(snowfall)+ scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df17)
result_df17 = model_df17.fit()

print(result_df17.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     2219.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:55:56   Log-Likelihood:                -12164.
No. Observations:               16381   AIC:                         2.441e+04
Df Residuals:                   16340   BIC:                         2.473e+04
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [9]:
anova_result_df17 = sm.stats.anova_lm(result_df17).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df17[anova_result_df17['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(tmin),1.0,1.211700,1.211700,4.675043,3.061840e-02
scale(windchill),1.0,1.481601,1.481601,5.716389,1.681851e-02
C(rain),1.0,2.262765,2.262765,8.730316,3.134086e-03
scale(sealevel),1.0,4.277806,4.277806,16.504850,4.875244e-05
scale(tmax),1.0,4.983769,4.983769,19.228635,1.166855e-05
scale(heat),1.0,13.008711,13.008711,50.190879,1.451852e-12
C(week7),6.0,211.958734,35.326456,136.298351,4.000273e-169
C(item_nbr),18.0,22766.218181,1264.789899,4879.877529,0.000000e+00
C(nothing),2.0,145092.097078,72546.048539,279900.900837,0.000000e+00


In [10]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df17.values, i) for i in range(df17.shape[1])]
vif["features"] = df17.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# C(week7) + C(item_nbr) 

,VIF Factor,features
0,1.041026,snowfall
1,1.052058,item_nbr
2,1.053852,store_nbr
3,1.079418,units
4,1.131880,resultdir
5,1.347866,preciptotal
6,1.502960,other
7,2.728118,weekend
8,2.734114,week7
9,8.657980,resultspeed


In [11]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
model_df17 = sm.OLS.from_formula('np.log1p(units) ~ C(week7) + C(item_nbr)  + 0', data = df17)
result_df17 = model_df17.fit()

print(result_df17.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     3671.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:56:51   Log-Likelihood:                -12224.
No. Observations:               16381   AIC:                         2.450e+04
Df Residuals:                   16356   BIC:                         2.469e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(week7)[0]            1.5076      0

In [12]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
X17 = df17[['week7', 'item_nbr']]
y17 = df17['units']
model17 = LinearRegression()

cv17 = KFold(n_splits=17, shuffle=True, random_state=0)

cross_val_score(model17, X17, y17, scoring="r2", cv=cv17)

array([0.03616173, 0.03996442, 0.02550267, 0.03253181, 0.0133985 ,
       0.02296577, 0.03945545, 0.01691954, 0.02630364, 0.02502422,
       0.03575539, 0.03814722, 0.02396514, 0.04958402, 0.03274111,
       0.0341533 , 0.00610493])